In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Table of Contents
<a id="toc"></a>
- [1. Introduction](#1)
- [2. Imports](#2)
- [3. Building the model from scratch](#3)
- [4. Method 1: Working with all the features in the data set](#4)
- [5. Visualizing how the model performs](#5)
- [6. Generating Predictions](#6)
- [7. Method 2: Working with only the important features obtained using RandomForestClassifier](#7)
- [8. Building the Neural Network Model](#8)
- [9. Generating Predictions](#9)


<a id="1"></a>
## 1. Introduction
**The task of this compeition is to classify 10 different bacteria species using data from a genomic analysis technique that has some data compression and data loss. The dataset used for this compeition is derived from this [paper](https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full).**

**Submissions are evaluated based on their categorization accuracy..**

- I am using Neural Network and also Random Forest for feature selection.

#### There are two ways to handle labels in multiclass classification:

- Encoding the labels via categorical encoding (also nown as one-hot-enoding) and using categorical_crossentropy as a loss function.

- Encoding the labels as integers and using the sparse_crossentropy as the loss function

<a id="2"></a>

## 2. Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
 
from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')


RANDOM_STATE = 12 
FOLDS = 5

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')


In [ ]:
data.shape, test.shape

In [ ]:
#sample_submission.head()

In [ ]:
#test.head()

In [ ]:
#data.head()

In [ ]:
data['target'].unique()

In [ ]:
print(data.shape, test.shape)

In [ ]:
train_labels = data['target']

In [ ]:
train_labels.head()

###### indexing not in column. No duplication detected if index are in column as indexes are unique 🙈
data= pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv", index_col="row_id")

duplicates_train = data.duplicated().sum()
print('Duplicates in train data: {0}'.format(duplicates_train))

data.drop_duplicates(keep='first', inplace=True)
duplicates_train = data.duplicated().sum()

print('Train data shape:', data.shape)
print('Duplicates in train data: {0}'.format(duplicates_train))

## standardizing the data

In [ ]:
useful_cols = [c for c in data.columns]


In [ ]:
useful_cols.remove('row_id')

In [ ]:
data = data[useful_cols]

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(keep='first', inplace=True)



In [ ]:
LABELS = pd.DataFrame()

y = data['target']
LABELS['target'] =y

In [ ]:
LABELS.head()

In [ ]:
LABELS.shape

In [ ]:
data.shape

In [ ]:
useful_col  = [c for c in data.columns]

In [ ]:
#useful_col

In [ ]:
useful_col.remove('target')

In [ ]:
data = data[useful_col]
print(data.shape)

In [ ]:
useful_col = [ c for c in data.columns]

In [ ]:

autoscaler = StandardScaler()
data[useful_col] = autoscaler.fit_transform(data[useful_col])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
cols_test = [c for c in test.columns]


In [ ]:
cols_test.remove('row_id')

In [ ]:
test = test[cols_test]

In [ ]:
test.shape

In [ ]:
cols_test  = [c for c in test.columns]

In [ ]:
test[cols_test] = autoscaler.transform(test[cols_test])

In [ ]:
display(data.head())
display(test.head())

<a id="3"></a>

## 3. Building the model from scratch

In [ ]:
from keras import models
from keras import layers

In [ ]:
#data.describe().T

In [ ]:
LABELS['target'].value_counts().plot(kind='bar')

print(LABELS['target'].value_counts())

## Label Encoding

In [ ]:
data = pd.concat([data, LABELS],axis=1)
data.head()

In [ ]:
data.shape

In [ ]:
le = LabelEncoder()
data["target"] = le.fit_transform(data["target"])
target = le.fit_transform(data.target)

display(data.head())
display(data.tail())

In [ ]:
cols = [c for c in data.columns]

In [ ]:
cols.remove('target')


In [ ]:
X =data[cols]

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y = data['target']

In [ ]:
y.head()

In [ ]:
# sklearn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 17171, stratify =y )

In [ ]:
X_train.shape, X_test.shape

<a id="4"></a>
## 4. Method 1: Working with all the features in the data set

In [ ]:
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

In [ ]:

model=Sequential()
# first layer 
# for initializing the weight we use he_normal
model.add(Dense(units=512,kernel_initializer='he_normal',activation='relu',input_dim=286))
#2nd layer
model.add(Dense(units=256,kernel_initializer='he_normal',kernel_regularizer = regularizers.l2(0.001),activation='relu'))
#3rd layer
model.add(Dense(units=64,kernel_initializer='he_normal',activation='relu'))
model.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
# to avoid overfitting we add dropout and kernel_regularizer
model.add(layers.Dropout(0.5))
model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
model.add(Dense(units = 64, kernel_initializer='he_normal',activation = 'relu'))
model.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))


In [ ]:
# compiling


model.compile(optimizer='Adamax',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

history = model.fit(X_train,y_train, epochs = 58, batch_size =512, 
                    validation_data = (X_test,y_test))

<a id="5"></a>

## 5. Visualizing how the model performs

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs,acc,'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

<a id="6"></a>

## 6. Generating Predictions

In [ ]:
pred = model.predict(test)

In [ ]:
len(pred)

In [ ]:
sample_submission['target'] = (np.argmax(pred, axis=1)) 
display(sample_submission.head(10))
display(sample_submission.tail(5))

In [ ]:

sample_submission = sample_submission.replace({'target':{0:'Bacteroides_fragilis',1:'Campylobacter_jejuni',2:'Enterococcus_hirae',
                3:'Escherichia_coli',4:'Escherichia_fergusonii',5:'Klebsiella_pneumoniae',6:'Salmonella_enterica',
                    7:'Staphylococcus_aureus',8:'Streptococcus_pneumoniae',9:'Streptococcus_pyogenes'}})

In [ ]:
sample_submission.head()

In [ ]:
sub = sample_submission.to_csv('sub1_all_NN.csv',index=False)

``` Public LB Score : 0.89724```

<a id="7"></a>
## 7. Method 2: Working with only the important features obtained using RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)


In [ ]:
print(selected_feat)

In [ ]:
df = data[selected_feat]
df.head()

In [ ]:
df.shape

In [ ]:
le = LabelEncoder()
data["target"] = le.fit_transform(data["target"])
target = le.fit_transform(data.target)


In [ ]:
X=df
y=data['target']

In [ ]:
#y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=17171, stratify = y)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
plt.figure(figsize=(10,6))
ax=y_train.value_counts().plot(kind="bar")
plt.show()

<a id="8"></a>

## 8. Building the Neural Network Model

In [ ]:

model2=Sequential()
# first layer 
# for initializing the weight we use he_normal
model2.add(Dense(units=512,kernel_initializer='he_normal',activation='relu',input_dim=len(selected_feat)))
#2nd layer
model2.add(Dense(units=256,kernel_initializer='he_normal',kernel_regularizer = regularizers.l2(0.001),activation='relu'))
#3rd layer
model2.add(Dense(units=64,kernel_initializer='he_normal',activation='relu'))
model2.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
model2.add(Dense(units = 64, kernel_initializer='he_normal',activation = 'relu'))
model2.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
model2.add(layers.Dropout(0.3))
model2.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
model2.add(Dense(units = 64, activation = 'relu'))
model2.add(Dense(10, activation = 'softmax'))



In [ ]:
# compiling


model2.compile(optimizer='Adamax',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history2 = model2.fit(X_train,y_train, epochs = 112, batch_size =512, 
                    validation_data = (X_test,y_test))

<a id=9></a>

## 9. Generating predictions 

In [ ]:
test_new2 = test[selected_feat]
pred = model2.predict(test_new2)

In [ ]:
sample_submission['target'] = (np.argmax(pred, axis=1)) 
display(sample_submission.head(10))
display(sample_submission.tail(5))


In [ ]:
sample_submission = sample_submission.replace({'target':{0:'Bacteroides_fragilis',1:'Campylobacter_jejuni',2:'Enterococcus_hirae',
                3:'Escherichia_coli',4:'Escherichia_fergusonii',5:'Klebsiella_pneumoniae',6:'Salmonella_enterica',
                    7:'Staphylococcus_aureus',8:'Streptococcus_pneumoniae',9:'Streptococcus_pyogenes'}})

In [ ]:
submission = sample_submission.to_csv('submission.csv',index=False)

``` Public LB score: 0.88896```